In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/local_Documents/TheoryAndPracticeOfCausalInference`


In [2]:
Pkg.add(["CSV", "DataFrames", "HTTP", "Statistics"])
using CSV, DataFrames, HTTP, Statistics

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [3]:
data06 = CSV.read(HTTP.get("https://raw.githubusercontent.com/mtakahashi123/causality/main/data06.csv").body, DataFrame; missingstring = "")

,country,y1,x1,x2
,String15,Float64,Float64,Float64
1,australia,3.87133,4.9,54.763
2,austria,15.6585,8.1,49.701
3,belgium,8.64357,5.6,46.198
4,brazil,0.0,1.2,8.755
5,canada,4.17387,4.0,46.55
6,china,0.0212498,0.1,10.004
7,denmark,22.2475,4.9,60.657
8,finland,7.26028,5.4,48.678
9,france,9.04682,4.3,40.319


In [4]:
function attach(x::DataFrame)
    for i in propertynames(x)
        @eval $(i) = $(x).$(i)
    end
end

attach (generic function with 1 method)

In [5]:
attach(data06)

In [6]:
describe(data06)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,country,,australia,,unitedstates,0,String15
2,y1,9.74774,0.0,8.64357,30.7631,0,Float64
3,x1,4.86957,0.1,4.9,8.8,0,Float64
4,x2,44.7045,8.755,46.232,85.135,0,Float64


In [7]:
n1 = nrow(data06)

23

## 分散

総和変動＝ノーベル賞受賞者数の分散

In [8]:
hensa = y1 .- mean(y1)
hensa2 = @. hensa ^ 2
tss = sum(hensa2) # 全体の平方和

1831.7182345412102

In [9]:
tss / (n1 - 1)

83.2599197518732

In [10]:
var(y1)

83.2599197518732

## ESSとUSS



In [11]:
Pkg.add(["GLM", "StatsPlots"])
using GLM, StatsPlots

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


`glm`ではなくて，`lm`というショートカットがあったらしい。

In [12]:
model1 = lm(@formula(y1 ~ x1), data06)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y1 ~ 1 + x1

Coefficients:
────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  -3.42165    3.22736   -1.06    0.3011  -10.1333     3.29002
x1            2.70443    0.598472   4.52    0.0002    1.45984    3.94902
────────────────────────────────────────────────────────────────────────

ESS (Explained SUm of Squares)

In [13]:
yhat1 = @. -3.4217 + 2.7044 * x1
yhat2 = (yhat1 .- mean(yhat1)) .^ 2
sum(yhat2)

903.0227978669911

In [14]:
predict(model1)

23-element Vector{Float64}:
  9.830048087206782
 18.48421880609108
 11.723147931962721
 -0.1763368065031905
  7.396062572520572
 -3.151207991119668
  9.830048087206782
 11.182262262032454
  8.207391077415975
 17.94333313616081
  3.3394200480435567
 17.94333313616081
  7.396062572520572
 -0.1763368065031905
 10.370933757137049
 12.26403360189299
 11.993590766927856
  1.9872058732178846
  7.396062572520572
 14.427576281614064
 20.37731865084702
 17.132004631265406
  8.47783391238111

Unexplained Sum of Squares (USS)

In [15]:
e1 = @. y1 - yhat1
e2 = e1 .^ 2
uss = sum(e2)

928.6765050876866

In [16]:
sum(residuals(model1) .^ 2)

928.676504182236

## 決定係数

1−TSSに占めるUSSの割合

In [17]:
1 - uss / tss

0.4930025330449954

In [18]:
r2(model1)

0.493002533539313

## 回帰係数の標準誤差と回帰の標準誤差

回帰係数のSE＝回帰の標準誤差を説明変数のスケールで標準化したもの。
回帰のSE＝USSを自由度で調整して，平方根をとったもの。

## 三変数の重回帰モデル

In [19]:
model1 = lm(@formula(x1 ~ x2), data06)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

x1 ~ 1 + x2

Coefficients:
────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  1.37876     0.874069   1.58    0.1296  -0.438961   3.19649
x2           0.0780862   0.0177668  4.40    0.0003   0.041138   0.115034
────────────────────────────────────────────────────────────────────────

In [20]:
ex1 = residuals(model1)

23-element Vector{Float64}:
 -0.7549964902925286
  2.84027561892396
  0.6138114147725107
 -0.8624087382513894
 -1.0136749112733607
 -2.0599383440221115
 -1.2152362792538067
  0.22015775399477633
 -0.2271200885693032
  2.911156485552171
 -0.4095654180004402
  0.14651621948938764
  0.03736471536708663
 -3.3071300332632134
 -0.4214691677401392
 -1.4341327733200027
  3.09317459927185
 -1.2020761491590672
  0.2930187820551131
  1.0907903583655472
  0.7733708544085331
  2.9529395795941653
 -2.064827988649742

In [21]:
model2 = lm(@formula(y1 ~ ex1), DataFrame(y1 = y1, ex1 = ex1))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y1 ~ 1 + ex1

Coefficients:
──────────────────────────────────────────────────────────────────────
               Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────
(Intercept)  9.74774     1.86839  5.22    <1e-04   5.86221     13.6333
ex1          1.50477     1.11734  1.35    0.1924  -0.818858     3.8284
──────────────────────────────────────────────────────────────────────

In [22]:
model3 = lm(@formula(y1 ~ x1 + x2), data06)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y1 ~ 1 + x1 + x2

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  -6.32035    3.20331    -1.97    0.0625  -13.0023      0.361644
x1            1.50477    0.756186    1.99    0.0604   -0.0726035   3.08215
x2            0.195517   0.0853061   2.29    0.0329    0.0175719   0.373463
───────────────────────────────────────────────────────────────────────────

In [23]:
confint(model3)

3×2 Matrix{Float64}:
 -13.0023     0.361644
  -0.0726035  3.08215
   0.0175719  0.373463

## 共分散分析（再考）

共分散分析によってATEが正しく推定できていたかどうかを確認する。

In [24]:
data03 = CSV.read(HTTP.get("https://raw.githubusercontent.com/mtakahashi123/causality/main/data03.csv").body, DataFrame; missingstring = "NA");

In [25]:
model1 = lm(@formula(t1 ~ x1), data03)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

t1 ~ 1 + x1

Coefficients:
────────────────────────────────────────────────────────────────────────
              Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)   2.74    1.07454     2.55    0.0201   0.482478   4.99752
x1           -0.028   0.0133666  -2.09    0.0506  -0.0560822  8.22373e-5
────────────────────────────────────────────────────────────────────────

In [26]:
insertcols!(data03, :et1 => residuals(model1));

In [27]:
model2 = lm(@formula(y3 ~ et1), data03)  # x1の交絡を取り除いてt1のみの効果を推定する。

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y3 ~ 1 + et1

Coefficients:
────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  77.25        1.2365   62.47    <1e-21   74.6522     79.8478
et1           9.81592     2.75801   3.56    0.0022    4.02156    15.6103
────────────────────────────────────────────────────────────────────────